<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Prophet_automatic_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import itertools
import numpy as np
import pandas as pd
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics

In [25]:
df = pd.read_csv('drive/My Drive/Churn/same_day_vol_port_basic_cdma_aug.csv')

In [26]:
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

In [27]:
# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = [] # Store the RMSEs for each params here

In [ ]:
# Use cross validation to evaluate all parameters
for params in all_params:
  m = Prophet(**params).fit(df) # Fit the model with given params
  df_cv = cross_validation(m, initial = '761 days', period = '180 days', horizon = '30 days')
  df_p = performance_metrics(df_cv, rolling_window = 1)
  rmses.append(df_p['rmse'].values[0])

In [ ]:
# Find the best results
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)
tuning_results.to_csv('drive/My Drive/Churn/same_day_vol_port_rmse.csv')

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)